In [1]:
#import dependencies
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import tensorflow as tf

In [2]:
#read in merged and cleaned data
df = pd.read_csv('../modified_data/pol_svi_sc_merged.csv')
df

,FIPS,3/31/21,AREA_SQMI,E_TOTPOP,E_HU,E_HH,E_POV,E_UNEMP,E_PCI,E_NOHSDP,...,Hopefulness,Income Per Capita,Neuroticism,Openness,Religiosity,Risk Taking,Selflessness,Tolerance,Work Ethic,dem_pct
0,1001,6589,594.443459,55200.0,23315.0,21115.0,8422.0,1065.0,29372.0,4204.0,...,91.163142,26168.0,77.925476,78.222354,91.106719,53.333333,82.142857,70.000000,60.380952,27.018365
1,1003,20505,1589.793007,208107.0,111945.0,78622.0,21653.0,4343.0,31203.0,14310.0,...,82.484017,28069.0,77.232120,80.086368,71.771566,67.272980,75.586018,66.983549,70.972246,22.409030
2,1005,2227,885.001636,25782.0,11937.0,9186.0,6597.0,918.0,18461.0,4901.0,...,61.927181,17249.0,80.375206,78.783778,73.657368,76.066481,78.753019,65.170377,68.704105,45.788173
3,1007,2542,622.461089,22527.0,9161.0,6840.0,2863.0,658.0,20199.0,2650.0,...,85.258871,18988.0,80.813736,77.837027,69.974652,75.136154,76.929754,69.859503,67.931677,20.698280
4,1009,6444,644.830460,57645.0,24222.0,20600.0,8220.0,909.0,22656.0,7861.0,...,79.492703,21033.0,78.764620,78.193105,92.045455,57.603815,79.307632,64.953288,76.000000,9.569378
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3053,56037,4022,10426.975725,44117.0,19628.0,15871.0,5237.0,1213.0,32624.0,2549.0,...,82.403142,30945.0,79.384759,79.347081,68.147062,73.938691,76.390464,67.420658,70.956334,22.894957
3054,56039,3609,3996.844622,23059.0,13680.0,9158.0,1619.0,210.0,53703.0,958.0,...,84.036899,46499.0,71.547359,80.522872,65.399695,79.598153,79.698193,70.877600,70.938645,66.599040
3055,56041,2128,2081.719807,20609.0,8972.0,7735.0,2552.0,614.0,27009.0,934.0,...,84.089095,25636.0,78.771570,77.859042,67.603416,69.705859,73.332067,67.404487,69.299391,16.819960
3056,56043,891,2238.672972,8129.0,3868.0,3422.0,984.0,253.0,27556.0,590.0,...,87.485019,26325.0,76.249370,77.658224,67.412774,82.820701,78.925326,74.628788,70.050103,16.145833


In [3]:
#set index to FIPS
df = df.set_index(df['FIPS'])
df= df.drop('FIPS', axis=1)

In [4]:
#rename target column
df = df.rename(columns={'3/31/21':'first_year_cases'})

In [5]:
#create variable for case% for each counties population
df['case_pct'] = df['first_year_cases']/df['E_TOTPOP']*100
df['case_pct'].head()

FIPS
1001    11.936594
1003     9.853104
1005     8.637809
1007    11.284237
1009    11.178767
Name: case_pct, dtype: float64

In [6]:
df['case_pct'].describe()

count    3058.000000
mean        9.426600
std         3.045809
min         0.000000
25%         7.713422
50%         9.466675
75%        11.176131
max        38.010657
Name: case_pct, dtype: float64

In [7]:
# bin and cut the case_pct column into 2 classifications
# q = df['case_pct'].quantile(.75)
q = df['case_pct'].quantile(.9)
bins = [0, 12 , 40]
labels = ['low','high']
df['case_class'] = pd.cut(df['case_pct'], bins, labels = labels)
df['case_class']

FIPS
1001      low
1003      low
1005      low
1007      low
1009      low
         ... 
56037     low
56039    high
56041     low
56043     low
56045     low
Name: case_class, Length: 3058, dtype: category
Categories (2, object): ['low' < 'high']

In [8]:
df['case_class'].value_counts()

low     2540
high     494
Name: case_class, dtype: int64

In [9]:
impact_cols = ['Empathy',
 'EPL_GROUPQ',
 'Agreeableness',
 'Employment Rate',
 'EPL_UNEMP',
 'Openness',
 'E_AGE65',
 'EP_AGE17',
 'Conscientiousness',
 'SPL_THEME4',
 'Income Per Capita',
 'E_POV',
 'first_year_cases',
 'E_MINRTY',
 'Conflict Awareness',
 'Extraversion',
 'EP_SNGPNT',
 'F_MOBILE',
 'Work Ethic',
 'E_AGE17',
 'EP_PCI',
 'Risk Taking',
 'E_UNEMP',
 'AREA_SQMI',
 'E_NOHSDP',
 'EPL_PCI',
 'E_HU',
 'E_NOVEH',
 'E_LIMENG',
 'EPL_SNGPNT',
 'E_UNINSUR',
 'EPL_AGE17',
 'Belief In Science',
 'EPL_LIMENG',
 'EPL_NOVEH',
 'Entrepreneurship',
 'SPL_THEME3',
 'E_HH',
 'EP_MUNIT',
 'EPL_MOBILE',
 'Collectivism',
 'RPL_THEME2',
 'E_MUNIT',
 'EPL_POV',
 'EP_UNINSUR',
 'dem_pct',
 'F_SNGPNT',
 'EPL_MUNIT',
 'EP_NOHSDP',
 'EPL_DISABL',
 'Hopefulness',
 'E_MOBILE']

In [10]:
# impact_cols_reduced = ['Empathy',
#  'EPL_GROUPQ',
#  'Agreeableness',
#  'Employment Rate',
#  'EPL_UNEMP',
#  'Openness',
#  'E_AGE65',
#  'EP_AGE17',
#  'Conscientiousness',
#  'SPL_THEME4',
#  'Income Per Capita',
#  'E_POV',
#  'first_year_cases',
#  'E_MINRTY',
#  'Conflict Awareness',
#  'Extraversion',
#  'EP_SNGPNT',
#  'F_MOBILE',
#  'Work Ethic',
#  'E_AGE17',
#  'EP_PCI',
#  'Risk Taking',
#  'E_UNEMP',
#  'AREA_SQMI',
#  'E_NOHSDP',
#  'EPL_PCI',
#  'E_HU',
#  'E_NOVEH',
#  'E_LIMENG',
#  'EPL_SNGPNT',
#  'E_UNINSUR',
#  'EPL_AGE17']

In [11]:
#drop unneeded columns
df = df.drop('case_pct', axis =1)


In [12]:
#turn case % classifications into binary 
df = pd.get_dummies(df, columns = ['case_class'])
df

,first_year_cases,AREA_SQMI,E_TOTPOP,E_HU,E_HH,E_POV,E_UNEMP,E_PCI,E_NOHSDP,E_AGE65,...,Neuroticism,Openness,Religiosity,Risk Taking,Selflessness,Tolerance,Work Ethic,dem_pct,case_class_low,case_class_high
FIPS,,,,,,,,,,,,,,,,,,,,,
1001,6589,594.443459,55200.0,23315.0,21115.0,8422.0,1065.0,29372.0,4204.0,8050.0,...,77.925476,78.222354,91.106719,53.333333,82.142857,70.000000,60.380952,27.018365,1,0
1003,20505,1589.793007,208107.0,111945.0,78622.0,21653.0,4343.0,31203.0,14310.0,40665.0,...,77.232120,80.086368,71.771566,67.272980,75.586018,66.983549,70.972246,22.409030,1,0
1005,2227,885.001636,25782.0,11937.0,9186.0,6597.0,918.0,18461.0,4901.0,4634.0,...,80.375206,78.783778,73.657368,76.066481,78.753019,65.170377,68.704105,45.788173,1,0
1007,2542,622.461089,22527.0,9161.0,6840.0,2863.0,658.0,20199.0,2650.0,3661.0,...,80.813736,77.837027,69.974652,75.136154,76.929754,69.859503,67.931677,20.698280,1,0
1009,6444,644.830460,57645.0,24222.0,20600.0,8220.0,909.0,22656.0,7861.0,10233.0,...,78.764620,78.193105,92.045455,57.603815,79.307632,64.953288,76.000000,9.569378,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
56037,4022,10426.975725,44117.0,19628.0,15871.0,5237.0,1213.0,32624.0,2549.0,4721.0,...,79.384759,79.347081,68.147062,73.938691,76.390464,67.420658,70.956334,22.894957,1,0
56039,3609,3996.844622,23059.0,13680.0,9158.0,1619.0,210.0,53703.0,958.0,3135.0,...,71.547359,80.522872,65.399695,79.598153,79.698193,70.877600,70.938645,66.599040,0,1
56041,2128,2081.719807,20609.0,8972.0,7735.0,2552.0,614.0,27009.0,934.0,2498.0,...,78.771570,77.859042,67.603416,69.705859,73.332067,67.404487,69.299391,16.819960,1,0


In [13]:
df['case_class_high'].value_counts()

0    2564
1     494
Name: case_class_high, dtype: int64

In [14]:
#seperate targets and features
## should i drop the number of cases?
X = df[impact_cols].values
y=df['case_class_high'].values

In [15]:
#split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)

In [16]:
# Create a StandardScaler instance
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [17]:
len(X_train[0])

52

In [18]:
# Define the model - deep neural net
number_input_features = len(X_train[0])
hidden_nodes_layer1 = 100
hidden_nodes_layer2 = 80
# hidden_nodes_layer1 = 100
# hidden_nodes_layer2 = 80
# hidden_nodes_layer3 = 80

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu")
)

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))
# nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer3, activation="relu"))
# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 100)               5300      
                                                                 
 dense_1 (Dense)             (None, 80)                8080      
                                                                 
 dense_2 (Dense)             (None, 1)                 81        
                                                                 


2022-08-08 18:51:29.195755: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Total params: 13,461
Trainable params: 13,461
Non-trainable params: 0
_________________________________________________________________


In [19]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [20]:
# Train the model
fit_model = nn.fit(X_train,y_train,epochs=150)

Epoch 1/150
72/72 [==============================] - 1s 3ms/step - loss: 498.2166 - accuracy: 0.7427
Epoch 2/150
72/72 [==============================] - 0s 3ms/step - loss: 129.0733 - accuracy: 0.7833
Epoch 3/150
72/72 [==============================] - 0s 3ms/step - loss: 152.1110 - accuracy: 0.7798
Epoch 4/150
72/72 [==============================] - 0s 4ms/step - loss: 124.0810 - accuracy: 0.8016
Epoch 5/150
72/72 [==============================] - 0s 4ms/step - loss: 189.9255 - accuracy: 0.8020
Epoch 6/150
72/72 [==============================] - 0s 3ms/step - loss: 52.7598 - accuracy: 0.8282
Epoch 7/150
72/72 [==============================] - 0s 3ms/step - loss: 36.8987 - accuracy: 0.8474
Epoch 8/150
72/72 [==============================] - 0s 3ms/step - loss: 92.8280 - accuracy: 0.8094
Epoch 9/150
72/72 [==============================] - 0s 3ms/step - loss: 87.3583 - accuracy: 0.8378
Epoch 10/150
72/72 [==============================] - 0s 3ms/step - loss: 334.5730 - accuracy: 

72/72 [==============================] - 0s 3ms/step - loss: 69.4451 - accuracy: 0.8770
Epoch 83/150
72/72 [==============================] - 0s 3ms/step - loss: 42.6792 - accuracy: 0.8386
Epoch 84/150
72/72 [==============================] - 0s 3ms/step - loss: 3.7168 - accuracy: 0.9036
Epoch 85/150
72/72 [==============================] - 0s 2ms/step - loss: 1.8326 - accuracy: 0.9263
Epoch 86/150
72/72 [==============================] - 0s 2ms/step - loss: 5.2690 - accuracy: 0.8940
Epoch 87/150
72/72 [==============================] - 0s 2ms/step - loss: 2.3271 - accuracy: 0.9246
Epoch 88/150
72/72 [==============================] - 0s 3ms/step - loss: 1.2134 - accuracy: 0.9350
Epoch 89/150
72/72 [==============================] - 0s 2ms/step - loss: 2.7531 - accuracy: 0.9145
Epoch 90/150
72/72 [==============================] - 0s 2ms/step - loss: 1.2460 - accuracy: 0.9368
Epoch 91/150
72/72 [==============================] - 0s 2ms/step - loss: 14.7767 - accuracy: 0.8596
Epoch 92/1

In [21]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

24/24 - 0s - loss: 35.0572 - accuracy: 0.8928 - 264ms/epoch - 11ms/step
Loss: 35.05717086791992, Accuracy: 0.8928104639053345


In [22]:
y_pred=nn.predict(X_test)
con_mat = tf.math.confusion_matrix(labels=y_test, predictions=y_pred)
con_mat

24/24 [==============================] - 0s 2ms/step


<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[625,   0],
       [124,  16]], dtype=int32)>

In [23]:
# nn.save('../aug_8_reduced_features_model.h5')

In [24]:
nn_imported = tf.keras.models.load_model('../aug_8_reduced_features_model.h5')

In [25]:
# Evaluate the completed model using the test data
model_loss, model_accuracy = nn_imported.evaluate(X_test,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

24/24 - 0s - loss: 15.6768 - accuracy: 0.9359 - 187ms/epoch - 8ms/step
Loss: 15.676847457885742, Accuracy: 0.9359477162361145


In [26]:
y_pred=nn_imported.predict(X_test)
con_mat = tf.math.confusion_matrix(labels=y_test, predictions=y_pred)
con_mat

24/24 [==============================] - 0s 3ms/step


<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[620,   5],
       [ 55,  85]], dtype=int32)>

In [33]:
pred = pd.DataFrame(y_pred)
pred

,0
0,5.073171e-01
1,1.001165e-20
2,1.000000e+00
3,6.239522e-07
4,0.000000e+00
...,...
760,9.410870e-07
761,0.000000e+00
762,0.000000e+00
763,4.267087e-07


In [34]:
pred.loc[pred[0]==1]

,0
2,1.0
16,1.0
22,1.0
36,1.0
43,1.0
...,...
722,1.0
725,1.0
727,1.0
738,1.0
